In [ ]:
import networkx as nx
import numpy as np
np.set_printoptions(suppress=True)
import scipy as sp
import matplotlib.pyplot as plt
from numpy.random import choice

from time import time

# 1. Preliminary parts

## 1.1 Epidemic on a known graph

In [ ]:
def simulate_epidemic_fixed_graph(G, beta=0.3, rho=0.7, n_weeks=15, n_initial_infected=10, n_simulations=100, random_infected_everytime=True):
    n = len(G)    # n. of people
    
    new_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_S = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_R = np.zeros((n_simulations, n_weeks+1), dtype=int)
    
    new_I[:,0] = n_initial_infected
    tot_S[:,0] = n-n_initial_infected
    tot_I[:,0] = n_initial_infected
    tot_R[:,0] = 0
    
    ### Week 1 (the epidemics breaks out)
    # Choose at random n_initial_infected people which are initially infected
    # (here or inside the simulations loop if random_infected_everytime)
    if not random_infected_everytime:
        initial_infected = choice(range(n), size=n_initial_infected, replace=False)
        initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
    
    
    for s in range(n_simulations):
        if random_infected_everytime:
            initial_infected = choice(range(n), size=n_initial_infected, replace=False)
            initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
        
        old_config = initial_config

        ### Weeks 2-(n_weeks-1) (the epidemics spreads)
        for w in range(0, n_weeks):
            
            new_config = []

            # Compute m for each node
            for i in range(n):

                if old_config[i] == "S":
                    #neighbors = [(i-2)%n, (i-1)%n, (i+1)%n, (i+2)%n]    # in exercise 1.1
                    neighbors = list(G.neighbors(i))
                    neighbors_state = old_config[neighbors]
                    m = sum([1 for st in neighbors_state if st=="I"])
                    p_infection = 1-(1-beta)**m
                    new_state = choice(("I","S"), p=(p_infection, 1-p_infection))
                    new_config.append(new_state)

                elif old_config[i] == "I":
                    new_state = choice(("R","I"), p=(rho, 1-rho))
                    new_config.append(new_state)

                else: #old_config[i] == "R"
                    new_config.append("R")
            
            # Compute the required quantities
            # Newly infected individuals
            new_I[s,w+1] = sum([1 for i in range(n) if (new_config[i]=="I" and old_config[i]!="I")])
            # Total n. of S,I,R individuals
            tot_S[s,w+1] = sum([1 for i in new_config if i=="S"])
            tot_I[s,w+1] = sum([1 for i in new_config if i=="I"])
            tot_R[s,w+1] = sum([1 for i in new_config if i=="R"])
            
            old_config = np.array(new_config)
        
    # Compute the required averages
    avg_new_I = np.mean(new_I, axis=0)
    avg_tot_S = np.mean(tot_S, axis=0)
    avg_tot_I = np.mean(tot_I, axis=0)
    avg_tot_R = np.mean(tot_R, axis=0)
    
    return (avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R)


In [ ]:
### Create 4-regular graph with n=500 nodes
GG = nx.Graph()
n_nodes = 500
nx.add_cycle(GG, range(n_nodes)) # C_n

for n in range(n_nodes):
    GG.add_edge(n,(n+1)%n_nodes)
    GG.add_edge(n,(n+2)%n_nodes)

#nx.draw_circular(GG, with_labels=False)


### Simulate epidemics
avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R = simulate_epidemic_fixed_graph(GG, beta=0.3, rho=0.7, n_weeks=15, n_initial_infected=10, n_simulations=100, random_infected_everytime=True)
print(avg_new_I)
print(avg_tot_S)
print(avg_tot_I)
print(avg_tot_R)


### Plots
n_weeks = len(avg_new_I)
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_new_I, label='Avg. newly infected', marker='.', markersize=15)
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
plt.tick_params(axis='both', labelsize=20)
ax.set_xlabel('Week', size='xx-large')
ax.set_ylabel('N. of individuals', size='xx-large')
ax.legend(prop={'size': 20})
plt.grid()

fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_tot_S, label='Avg. total susceptible', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_tot_I, label='Avg. total infected', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_tot_R, label='Avg. total recovered', marker='.', markersize=15)
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
plt.tick_params(axis='both', labelsize=20)
ax.set_xlabel('Week', size='xx-large')
ax.set_ylabel('N. of individuals', size='xx-large')
ax.legend(prop={'size': 20})
plt.grid()

## 1.2 Generate a random graph

In [ ]:
def generate_preferential_attachment_graph(k=4, a=0, n_nodes=1000, seed=False):
    # a = intrinsic probability of a node to be selected as a neighbor from new nodes
    # k -> (k+1)-complete graph and floor(k/2) or ceil(k/2) links added at each step (for the node that is added on that step)
    # n_nodes = number of nodes
    # seed = the seed to set np.random.seed to, and in this case returns the same graph (fixed k and a);
    #        if False, then do not set it, and so returns always a different graph
    
    np.random.seed()
    if seed:
        np.random.seed(42)    # the same graph will be generated for the same k, a, n_nodes
    
    # PAG is initialized to G1, a complete graph with k+1 nodes (nodes: 0,1,...,k)
    PAG = nx.complete_graph(k+1)

    # Compute n_iterations
    n_iterations = n_nodes - len(PAG)

    node_to_add = k+1

    for t in range(2,n_iterations+2):
        ### step t is beginning

        # Add a new node to PAG
        PAG.add_node(node_to_add)

        # Connect the new node to c=k/2 existing nodes
        degrees = np.array([d for _, d in PAG.degree()])
        deg_distr = (degrees+a)/np.sum(degrees+a)

        # Choose the value of c, i.e. how many existing nodes the new node must be connected to
        if t%2 == 0:
            c = int(np.floor(k/2))
        else:
            c = int(np.ceil(k/2))

        neighbors = choice(np.arange(len(PAG)), size=c, p=deg_distr, replace=False)    # replace=False guarantees no neighbor is chosen twice
        for neigh in neighbors:
            PAG.add_edge(node_to_add,neigh)
            
        node_to_add += 1

    return PAG


In [ ]:
GPA = generate_preferential_attachment_graph(k=4, a=0, n_nodes=1000, seed=True)
avg_degree = np.mean([d for _,d in GPA.degree()])
print(avg_degree)

# 2. Simulate a pandemic without vaccination

In [ ]:
def simulate_epidemic_without_vaccination(n, k, a, beta=0.3, rho=0.7, n_weeks=15, n_initial_infected=10, n_simulations=100, random_infected_everytime=True, gen_rand_graph_everytime=True, seed=False):
    # random_infected_everytime: if True, each simulation will start with a different set of initially infected nodes
    # gen_rand_graph_everytime: if True, each simulation will be on a different random graph
    # seed: if True, generate a single random graph for all the simulations (moreover, same k and a always implies same graph)
    
    np.random.seed()
    
    new_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_S = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_R = np.zeros((n_simulations, n_weeks+1), dtype=int)
    
    ### Beginning of week 1
    # Initial situation
    new_I[:,0] = n_initial_infected
    tot_S[:,0] = n-n_initial_infected
    tot_I[:,0] = n_initial_infected
    tot_R[:,0] = 0
    
    # Choose at random n_initial_infected people which are initially infected
    # (here or inside the simulations loop if random_infected_everytime)
    if not random_infected_everytime:
        initial_infected = choice(range(n), size=n_initial_infected, replace=False)
        initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
    
    if not gen_rand_graph_everytime:
        # Generate a unique random graph (either with seed or not)
        # moreover, if seed=True, the unique random graph is always the same, under the same k and a
        # else, the unique random graph is always different
        G = generate_preferential_attachment_graph(k=k, a=a, n_nodes=n, seed=seed)
        np.random.seed()
    
    
    for s in range(n_simulations):
        
        if gen_rand_graph_everytime:
            # Generate the i-th random graph for the i-th simulation (max degree of randomness)
            G = generate_preferential_attachment_graph(k=k, a=a, n_nodes=n, seed=False)
        
        if random_infected_everytime:
            initial_infected = choice(range(n), size=n_initial_infected, replace=False)
            initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
        
        old_config = initial_config

        ### Weeks 1-(n_weeks-1) (the epidemics spreads)
        for w in range(0, n_weeks):
            
            new_config = []
            
            count_new_I = 0
            count_tot_S = 0
            count_tot_I = 0
            count_tot_R = 0

            # Compute m for each node
            for i in range(n):

                if old_config[i] == "S":
                    #neighbors = [(i-2)%n, (i-1)%n, (i+1)%n, (i+2)%n]    # in exercise 1.1
                    neighbors = list(G.neighbors(i))
                    neighbors_state = old_config[neighbors]
                    m = sum([1 for st in neighbors_state if st=="I"])
                    p_infection = 1-(1-beta)**m
                    new_state = choice(("I","S"), p=(p_infection, 1-p_infection))
                    new_config.append(new_state)
                    if new_state == "I":    # update counter
                        count_new_I += 1
                        count_tot_I += 1
                    else:
                        count_tot_S += 1

                elif old_config[i] == "I":
                    new_state = choice(("R","I"), p=(rho, 1-rho))
                    new_config.append(new_state)
                    if new_state == "R":    # update counter
                        count_tot_R += 1
                    else:
                        count_tot_I += 1

                else: #old_config[i] == "R"
                    new_config.append("R")
                    count_tot_R += 1
            
            # Compute the required quantities
            # Newly infected individuals
            new_I[s,w+1] = count_new_I    # i.e. during week w, count_new_I people were infected
            # Total n. of S,I,R individuals
            tot_S[s,w+1] = count_tot_S    # i.e. at the beginning of week w+1, count_tot_S people were "S"
            tot_I[s,w+1] = count_tot_I    # i.e. at the beginning of week w+1, count_tot_I people were "I"
            tot_R[s,w+1] = count_tot_R    # i.e. at the beginning of week w+1, count_tot_R people were "R" (equivalent to say that a total of count_tot_R people recovered)
            
            old_config = np.array(new_config)
        
    # Compute the required averages
    avg_new_I = np.mean(new_I, axis=0)
    avg_tot_S = np.mean(tot_S, axis=0)
    avg_tot_I = np.mean(tot_I, axis=0)
    avg_tot_R = np.mean(tot_R, axis=0)
    
    return (avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R)


In [ ]:
### Simulate epidemics
avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R = simulate_epidemic_without_vaccination(500, 6, 0, beta=0.3, rho=0.7, n_weeks=15, n_initial_infected=10, n_simulations=100, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=True)

print(avg_new_I)
print(avg_tot_S)
print(avg_tot_I)
print(avg_tot_R)

### Plots
n_weeks = len(avg_new_I)
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_new_I, label='Avg. newly infected', marker='.', markersize=15)
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
plt.tick_params(axis='both', labelsize=20)
ax.set_xlabel('Week', size='xx-large')
ax.set_ylabel('N. of individuals', size='xx-large')
ax.legend(prop={'size': 20})
plt.grid()

fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_tot_S, label='Avg. total susceptible', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_tot_I, label='Avg. total infected', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_tot_R, label='Avg. total recovered', marker='.', markersize=15)
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
plt.tick_params(axis='both', labelsize=20)
ax.set_xlabel('Week', size='xx-large')
ax.set_ylabel('N. of individuals', size='xx-large')
ax.legend(prop={'size': 20})
plt.grid()

# 3. Simulate a pandemic with vaccination

In [ ]:
def simulate_epidemic_with_vaccination(n, k, a, vacc, beta=0.3, rho=0.7, n_weeks=15, n_initial_infected=10, n_simulations=100, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=True):
    # random_infected_everytime: if True, each simulation will start with a different set of initially infected nodes
    # gen_rand_graph_everytime: if True, each simulation will be on a different random graph
    # seed: if True, generate a single random graph for all the simulations (moreover, same k and a always implies same graph)
    
    np.random.seed()
    
    new_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_S = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_R = np.zeros((n_simulations, n_weeks+1), dtype=int)
    
    # Compute how many people get vaccinated by each week
    # new_V[i] is the n. of people who get vaccinated during week i-1 (i=0 => during the previous weeks)
    # (N.B: for simplicity, we pretend to vaccinate them all at the beginning of week i)
    # tot_V[i] is the n. of people who are "V" at the beginning of week i
    tot_V = ((np.array(vacc) / 100) * n)
    new_V = np.array([tot_V[i]-tot_V[i-1] if i!=0 else tot_V[0] for i in range(len(vacc))])
    
    ### Beginning of week 1
    # Initial situation
    new_I[:,0] = n_initial_infected    # i.e. in the weeks before the starting week (week 1), a total of n_initial_infected people were "I"
    tot_S[:,0] = n-n_initial_infected    # i.e. in the weeks before the starting week (week 1), a total of n-n_initial_infected people were "S"
    tot_I[:,0] = n_initial_infected    # i.e. in the weeks before the starting week (week 1), a total of n_initial_infected people were "I"
    tot_R[:,0] = 0    # i.e. in the weeks before the starting week (week 1), no one was "R"
    
    # Choose at random n_initial_infected people which are initially infected
    # (here or inside the simulations loop if random_infected_everytime)
    if not random_infected_everytime:
        initial_infected = choice(range(n), size=n_initial_infected, replace=False)
        initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
    
    if not gen_rand_graph_everytime:
        # Generate a unique random graph (either with seed or not)
        # moreover, if seed=True, the unique random graph is always the same, under the same k and a
        # else, the unique random graph is always different
        G = generate_preferential_attachment_graph(k=k, a=a, n_nodes=n, seed=seed)
        np.random.seed()
    
    
    for s in range(n_simulations):
        
        count_tot_R = 0    # count_tot_R is increased by 1 only when an I becomes R
        
        if gen_rand_graph_everytime:
            # Generate the i-th random graph for the i-th simulation (max degree of randomness)
            G = generate_preferential_attachment_graph(k=k, a=a, n_nodes=n, seed=False)
        
        if random_infected_everytime:
            initial_infected = choice(range(n), size=n_initial_infected, replace=False)
            initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
        
        # Vaccination: initial situation
        non_vacc_people = set(range(n))
        
        old_config = initial_config

        ### Weeks 1-n_weeks (the epidemics spreads)
        for w in range(0, n_weeks):    # week w=0 means the 1st week and so on (mapping from 0...15 to 1...16)
            # We want to simulate week w; the final configuration is the starting configuration of week w+1
            
            new_config = []
            
            count_new_I = 0
            count_tot_S = 0
            count_tot_I = 0
            
            # At the beginning of week w, we take into account the people vaccinated during week w-1
            # (for simplicity, we pretend to vaccinate them all at the beginning of week w)
            n_to_vacc = int(new_V[w])
            new_vaccinated = choice(list(non_vacc_people), size=n_to_vacc, replace=False)
            non_vacc_people -= set(new_vaccinated)
            old_config[new_vaccinated] = "V"
            count_new_V = n_to_vacc
            
            for i in range(n):
                
                if old_config[i] == "V":
                    new_config.append("V")
                    
                elif old_config[i] == "S":
                    neighbors = list(G.neighbors(i))
                    neighbors_state = old_config[neighbors]
                    m = np.sum(neighbors_state == "I")
                    p_infection = 1-(1-beta)**m
                    new_state = choice(("I","S"), p=(p_infection, 1-p_infection))
                    new_config.append(new_state)
                    if new_state == "I":    # update counter
                        count_new_I += 1
                        count_tot_I += 1
                    else:
                        count_tot_S += 1

                elif old_config[i] == "I":
                    new_state = choice(("R","I"), p=(rho, 1-rho))
                    new_config.append(new_state)
                    if new_state == "R":    # update counter
                        count_tot_R += 1
                    else:
                        count_tot_I += 1

                else: #old_config[i] == "R"
                    new_config.append("R")
            
            # Compute the required quantities
            # Newly infected individuals
            new_I[s,w+1] = count_new_I    # i.e. during week w, count_new_I people were infected
            # Total n. of S,I,R individuals
            tot_S[s,w+1] = count_tot_S    # i.e. at the beginning of week w+1, count_tot_S people were "S"
            tot_I[s,w+1] = count_tot_I    # i.e. at the beginning of week w+1, count_tot_I people were "I"
            tot_R[s,w+1] = count_tot_R    # i.e. at the beginning of week w+1, a total of count_tot_R people recovered
            
            old_config = np.array(new_config)
        
    # Compute the required averages
    avg_new_I = np.mean(new_I, axis=0)
    avg_new_V = new_V
    avg_tot_S = np.mean(tot_S, axis=0)
    avg_tot_I = np.mean(tot_I, axis=0)
    avg_tot_R = np.mean(tot_R, axis=0)
    avg_tot_V = tot_V
    
    return (avg_new_I, avg_new_V, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V)


In [ ]:
### Simulate epidemics
vacc = [0, 5, 15, 25, 35, 45, 55, 60, 60, 60, 60, 60, 60, 60, 60, 60]
avg_new_I, avg_new_V, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = simulate_epidemic_with_vaccination(500, 6, 0, vacc, beta=0.3, rho=0.7, n_weeks=len(vacc)-1, n_initial_infected=10, n_simulations=100, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=True)

print(avg_new_I)
print(avg_new_V)
print(avg_tot_S)
print(avg_tot_I)
print(avg_tot_R)
print(avg_tot_V)

### Plots
n_weeks = len(avg_new_I)
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_new_I, label='Avg. newly infected', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_new_V, label='Avg. newly vaccinated', marker='.', markersize=15)
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
plt.tick_params(axis='both', labelsize=20)
ax.set_xlabel('Week', size='xx-large')
ax.set_ylabel('N. of individuals', size='xx-large')
ax.legend(prop={'size': 20})
plt.grid()

fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_tot_S, label='Avg. total susceptible', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_tot_I, label='Avg. total infected', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_tot_R, label='Avg. total recovered', marker='.', markersize=15)
ax.plot(range(n_weeks), avg_tot_V, label='Avg. total vaccinated', marker='.', markersize=15)
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
plt.tick_params(axis='both', labelsize=20)
ax.set_xlabel('Week', size='xx-large')
ax.set_ylabel('N. of individuals', size='xx-large')
ax.legend(prop={'size': 20})
plt.grid()


# 4. The H1N1 pandemic in Sweden 2009

In [ ]:
def find_best_k_beta_rho(weekly_infected, vacc, n_nodes, k0, beta0, rho0, dk, dbeta, drho, a=0, n_simulations=10, seed=True):
    
    initial_beta0 = beta0
    initial_rho0 = rho0
    
    refinement_done = False
    
    print("Initial parameters:", k0, beta0, rho0)
    n_weeks = len(weekly_infected)-1
    n_initial_infected = weekly_infected[0]
    
    while True:
        parameter_space = np.array(np.meshgrid([k0-dk,k0,k0+dk], [beta0-dbeta,beta0,beta0+dbeta], [rho0-drho,rho0,rho0+drho])).T.reshape(-1,3)

        RMSE = []
        avg_new_Is = []
        avg_tot_Ss = []
        avg_tot_Is = []
        avg_tot_Rs = []
        avg_tot_V = np.array(vacc)

        for param in parameter_space:
            k = int(param[0])
            beta = param[1]
            rho = param[2]
            
            avg_new_I, _, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = simulate_epidemic_with_vaccination(n_nodes, k, a, vacc, beta=beta, rho=rho, n_weeks=n_weeks, n_initial_infected=n_initial_infected, n_simulations=n_simulations, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=seed)
            avg_new_Is.append(avg_new_I)
            avg_tot_Ss.append(avg_tot_S)
            avg_tot_Is.append(avg_tot_I)
            avg_tot_Rs.append(avg_tot_R)
            
            RMSE.append( np.sqrt((1/(n_weeks+1))*np.sum((avg_new_I-weekly_infected)**2)) )
            
        # Now that all the 27 RMSEs have been computed
        new_param_min_RMSE = parameter_space[np.argmin(RMSE)]
        
        best_avg_new_I = avg_new_Is[np.argmin(RMSE)]
        best_avg_tot_S = avg_tot_Ss[np.argmin(RMSE)]
        best_avg_tot_I = avg_tot_Is[np.argmin(RMSE)]
        best_avg_tot_R = avg_tot_Rs[np.argmin(RMSE)]
        
        ### Stop criterion
        if np.all( new_param_min_RMSE == (k0, beta0, rho0) ):
            # Do a refinement of the deltas
            if not refinement_done:
                dbeta /= 2
                drho /= 2
                refinement_done = True
                continue
            print(f"A local minimum has been reached! RMSE = {np.min(RMSE)}, params = {new_param_min_RMSE}")
            return (best_avg_new_I, best_avg_tot_S, best_avg_tot_I, best_avg_tot_R, avg_tot_V)
        else:
            print(f"Better params found. Best RMSE = {np.min(RMSE)}, new params = {new_param_min_RMSE}")
            k0, beta0, rho0 = new_param_min_RMSE


In [ ]:
vacc = [5, 9, 16, 24, 32, 40, 47, 54, 59, 60, 60, 60, 60, 60, 60, 60]
weekly_infected = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0, 0]
n_simulations = 30
avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = find_best_k_beta_rho(weekly_infected, vacc, a=0, n_nodes=934, k0=10, beta0=0.3, rho0=0.6, dk=1, dbeta=0.1, drho=0.1, n_simulations=n_simulations, seed=True)

### Plots
n_weeks = len(vacc)
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_new_I, label='Avg. newly infected (model with best params)')
ax.plot(range(n_weeks), weekly_infected, label='Actual n. of newly infected individuals each week')
plt.setp( ax, xticks=range(n_weeks), xticklabels=list(range(42,53))+list(range(1,6)) )
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()

fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_tot_S, label='Avg. total susceptible')
ax.plot(range(n_weeks), avg_tot_I, label='Avg. total infected')
ax.plot(range(n_weeks), avg_tot_R, label='Avg. total recovered')
ax.plot(range(n_weeks), avg_tot_V, label='Avg. total vaccinated')
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()

# 5. Challenge

## Erdos-Renyi

In [ ]:
def generate_erdos_renyi_graph(n, p, seed=False):

    # Add links between couple of different nodes with probability p
    np.random.seed()
    if seed:
        np.random.seed(42)    # the same graph will be generated for the same k, a, n_nodes

    WER = np.random.choice([0,1], size=(n, n), p=[1-p,p])    # random weight (adjacency) matrix

    # GER has no self loops and is undirected (so WER must be symmetric)
    
    for i in range(n):
        WER[i,i] = 0    # no self loops
        WER[i+1:,i] = WER[i,i+1:]    # to make the WER symmetric
    GER = nx.from_numpy_array(WER, create_using=nx.Graph)
    return GER


In [ ]:
def simulate_epidemic_with_vaccination_ER(n, p, vacc, beta=0.3, rho=0.7, n_weeks=15, n_initial_infected=10, n_simulations=100, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=True):
    
    np.random.seed()
    
    new_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_S = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_I = np.zeros((n_simulations, n_weeks+1), dtype=int)
    tot_R = np.zeros((n_simulations, n_weeks+1), dtype=int)
    
    tot_V = ((np.array(vacc) / 100) * n)
    new_V = np.array([tot_V[i]-tot_V[i-1] if i!=0 else tot_V[0] for i in range(len(vacc))])
    
    new_I[:,0] = n_initial_infected
    tot_S[:,0] = n-n_initial_infected
    tot_I[:,0] = n_initial_infected
    tot_R[:,0] = 0
    
    if not random_infected_everytime:
        initial_infected = choice(range(n), size=n_initial_infected, replace=False)
        initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
    
    if not gen_rand_graph_everytime:
        G = generate_erdos_renyi_graph(n=n, p=p, seed=seed)
    
    
    for s in range(n_simulations):
        
        count_tot_R = 0
        
        if gen_rand_graph_everytime:
            G = generate_erdos_renyi_graph(n, p, seed=False)
            np.random.seed()
        
        if random_infected_everytime:
            initial_infected = choice(range(n), size=n_initial_infected, replace=False)
            initial_config = np.array(["I" if i in initial_infected else "S" for i in range(n)])
        
        non_vacc_people = set(range(n))
        
        old_config = initial_config
        
        for w in range(0, n_weeks):
            
            new_config = []
            
            count_new_I = 0
            count_tot_S = 0
            count_tot_I = 0
            
            n_to_vacc = int(new_V[w])
            new_vaccinated = choice(list(non_vacc_people), size=n_to_vacc, replace=False)
            non_vacc_people -= set(new_vaccinated)
            old_config[new_vaccinated] = "V"
            count_new_V = n_to_vacc
            
            for i in range(n):
                
                if old_config[i] == "V":
                    new_config.append("V")
                    
                elif old_config[i] == "S":
                    neighbors = list(G.neighbors(i))
                    neighbors_state = old_config[neighbors]
                    m = np.sum(neighbors_state == "I")
                    p_infection = 1-(1-beta)**m
                    new_state = choice(("I","S"), p=(p_infection, 1-p_infection))
                    new_config.append(new_state)
                    if new_state == "I":
                        count_new_I += 1
                        count_tot_I += 1
                    else:
                        count_tot_S += 1

                elif old_config[i] == "I":
                    new_state = choice(("R","I"), p=(rho, 1-rho))
                    new_config.append(new_state)
                    if new_state == "R":
                        count_tot_R += 1
                    else:
                        count_tot_I += 1

                else:
                    new_config.append("R")
                    
            new_I[s,w+1] = count_new_I
            tot_S[s,w+1] = count_tot_S
            tot_I[s,w+1] = count_tot_I
            tot_R[s,w+1] = count_tot_R
            
            old_config = np.array(new_config)
        
    avg_new_I = np.mean(new_I, axis=0)
    avg_new_V = new_V
    avg_tot_S = np.mean(tot_S, axis=0)
    avg_tot_I = np.mean(tot_I, axis=0)
    avg_tot_R = np.mean(tot_R, axis=0)
    avg_tot_V = tot_V
    
    return (avg_new_I, avg_new_V, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V)


In [ ]:
def find_best_k_beta_rho_ER(weekly_infected, vacc, n_nodes, k0, beta0, rho0, dk, dbeta, drho, n_simulations=10, seed=True):
    
    initial_beta0 = beta0
    initial_rho0 = rho0
    
    refinement_done = False
    
    print("Initial parameters:", k0, beta0, rho0)
    counter = 0
    n_weeks = len(weekly_infected)-1
    n_initial_infected = weekly_infected[0]
    
    while True:
        parameter_space = np.array(np.meshgrid([k0-dk,k0,k0+dk], [beta0-dbeta,beta0,beta0+dbeta], [rho0-drho,rho0,rho0+drho])).T.reshape(-1,3)

        RMSE = []
        avg_new_Is = []
        avg_tot_Ss = []
        avg_tot_Is = []
        avg_tot_Rs = []
        avg_tot_V = np.array(vacc)
        counter += 1

        for param in parameter_space:
            k = int(param[0])
            beta = param[1]
            rho = param[2]
            
            p = k/(n_nodes-1)    # this value of p ensures a mean degree of k
            
            avg_new_I, _, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = simulate_epidemic_with_vaccination_ER(n_nodes, p, vacc, beta=beta, rho=rho, n_weeks=n_weeks, n_initial_infected=n_initial_infected, n_simulations=n_simulations, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=seed)
            avg_new_Is.append(avg_new_I)
            avg_tot_Ss.append(avg_tot_S)
            avg_tot_Is.append(avg_tot_I)
            avg_tot_Rs.append(avg_tot_R)
            
            RMSE.append( np.sqrt((1/(n_weeks+1))*np.sum((avg_new_I-weekly_infected)**2)) )
            
        # Now that all the 27 RMSEs have been computed
        new_param_min_RMSE = parameter_space[np.argmin(RMSE)]
        
        best_avg_new_I = avg_new_Is[np.argmin(RMSE)]
        best_avg_tot_S = avg_tot_Ss[np.argmin(RMSE)]
        best_avg_tot_I = avg_tot_Is[np.argmin(RMSE)]
        best_avg_tot_R = avg_tot_Rs[np.argmin(RMSE)]
        
        ### Stop criterion
        if np.all( new_param_min_RMSE == (k0, beta0, rho0) ):
            # Do a refinement of the deltas
            if not refinement_done:
                dbeta /= 2
                drho /= 2
                refinement_done = True
                continue
            print(f"A local minimum has been reached! RMSE = {np.min(RMSE)}, params = {new_param_min_RMSE}")
            return (best_avg_new_I, best_avg_tot_S, best_avg_tot_I, best_avg_tot_R, avg_tot_V)
        else:
            print(f"Better params found. Best RMSE = {np.min(RMSE)}, new params = {new_param_min_RMSE}")
            k0, beta0, rho0 = new_param_min_RMSE


In [ ]:
vacc = [5, 9, 16, 24, 32, 40, 47, 54, 59, 60, 60, 60, 60, 60, 60, 60]
weekly_infected = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0, 0]
n_simulations = 30
avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = find_best_k_beta_rho_ER(weekly_infected, vacc, n_nodes=934, k0=15, beta0=0.3, rho0=0.6, dk=1, dbeta=0.1, drho=0.1, n_simulations=n_simulations, seed=True)

### Plots
n_weeks = len(vacc)
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_new_I, label='Avg. newly infected (model with best params)')
ax.plot(range(n_weeks), weekly_infected, label='Actual n. of newly infected individuals each week')
plt.setp( ax, xticks=range(n_weeks), xticklabels=list(range(42,53))+list(range(1,6)) )
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()

fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_tot_S, label='Avg. total susceptible')
ax.plot(range(n_weeks), avg_tot_I, label='Avg. total infected')
ax.plot(range(n_weeks), avg_tot_R, label='Avg. total recovered')
ax.plot(range(n_weeks), avg_tot_V, label='Avg. total vaccinated')
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()

## Grid search

In [ ]:
def grid_search_k_beta_rho(weekly_infected, vacc, n_nodes, k_list, beta_list, rho_list, a=0, n_simulations=10, seed=True):
    
    n_weeks = len(weekly_infected)-1
    n_initial_infected = weekly_infected[0]
    
    RMSE = []
    avg_new_Is = []
    avg_tot_Ss = []
    avg_tot_Is = []
    avg_tot_Rs = []
    avg_tot_V = np.array(vacc)
    
    parameter_space = np.array(np.meshgrid(k_list, beta_list, rho_list)).T.reshape(-1,3)
    
    for i,param in enumerate(parameter_space):
        k = int(param[0])
        beta = param[1]
        rho = param[2]
        print(f"Now testing: ({k},{beta},{rho}), trial {i+1} out of {len(parameter_space)}")
            
        avg_new_I, _, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = simulate_epidemic_with_vaccination(n_nodes, k, a, vacc, beta=beta, rho=rho, n_weeks=n_weeks, n_initial_infected=n_initial_infected, n_simulations=n_simulations, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=seed)
        avg_new_Is.append(avg_new_I)
        avg_tot_Ss.append(avg_tot_S)
        avg_tot_Is.append(avg_tot_I)
        avg_tot_Rs.append(avg_tot_R)
            
        RMSE.append( np.sqrt((1/(n_weeks+1))*np.sum((avg_new_I-weekly_infected)**2)) )
            
    # Now that all the RMSEs have been computed
    new_param_min_RMSE = parameter_space[np.argmin(RMSE)]
        
    best_avg_new_I = avg_new_Is[np.argmin(RMSE)]
    best_avg_tot_S = avg_tot_Ss[np.argmin(RMSE)]
    best_avg_tot_I = avg_tot_Is[np.argmin(RMSE)]
    best_avg_tot_R = avg_tot_Rs[np.argmin(RMSE)]
        
    print(f"All the sets have been tested! RMSE = {np.min(RMSE)}, params = {new_param_min_RMSE}")
    return (best_avg_new_I, best_avg_tot_S, best_avg_tot_I, best_avg_tot_R, avg_tot_V)


In [ ]:
vacc = [5, 9, 16, 24, 32, 40, 47, 54, 59, 60, 60, 60, 60, 60, 60, 60]
weekly_infected = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0, 0]
n_simulations = 30

k_list = [7,8,9,10,11,12,13,14,15,16]
beta_list = [0.05,0.1,0.15,0.2]
rho_list = [0.6,0.7,0.8,0.9]
avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = grid_search_k_beta_rho(weekly_infected, vacc, a=0, n_nodes=934, k_list=k_list, beta_list=beta_list, rho_list=rho_list, n_simulations=n_simulations, seed=True)

### Plots
n_weeks = len(vacc)
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_new_I, label='Avg. newly infected (model with best params)')
ax.plot(range(n_weeks), weekly_infected, label='Actual n. of newly infected individuals each week')
plt.setp( ax, xticks=range(n_weeks), xticklabels=list(range(42,53))+list(range(1,6)) )
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()

fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_tot_S, label='Avg. total susceptible')
ax.plot(range(n_weeks), avg_tot_I, label='Avg. total infected')
ax.plot(range(n_weeks), avg_tot_R, label='Avg. total recovered')
ax.plot(range(n_weeks), avg_tot_V, label='Avg. total vaccinated')
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()

## Random search

In [ ]:
def random_search_k_beta_rho(weekly_infected, vacc, n_nodes, k_range=list(range(7,17)), beta_range=(0.03,0.25), rho_range=(0.6,0.95), n_rand_trials=100, a=0, n_simulations=10, seed=True):
    
    n_weeks = len(weekly_infected)-1
    n_initial_infected = weekly_infected[0]
    
    RMSE = []
    parameters = []
    avg_new_Is = []
    avg_tot_Ss = []
    avg_tot_Is = []
    avg_tot_Rs = []
    avg_tot_V = np.array(vacc)
    
    for i in range(n_rand_trials):
        k = int(np.random.choice(k_range, size=1))
        beta = np.random.uniform(beta_range[0],beta_range[1],1)[0]
        rho = np.random.uniform(rho_range[0],rho_range[1],1)[0]
        print(f"Now testing: ({k},{beta},{rho}), trial {i+1} out of {n_rand_trials}")
            
        avg_new_I, _, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = simulate_epidemic_with_vaccination(n_nodes, k, a, vacc, beta=beta, rho=rho, n_weeks=n_weeks, n_initial_infected=n_initial_infected, n_simulations=n_simulations, random_infected_everytime=True, gen_rand_graph_everytime=False, seed=seed)
        avg_new_Is.append(avg_new_I)
        avg_tot_Ss.append(avg_tot_S)
        avg_tot_Is.append(avg_tot_I)
        avg_tot_Rs.append(avg_tot_R)
            
        RMSE.append( np.sqrt((1/(n_weeks+1))*np.sum((avg_new_I-weekly_infected)**2)) )
        parameters.append((k,beta,rho))
            
    # Now that all the RMSEs have been computed
    param_min_RMSE = parameters[np.argmin(RMSE)]
        
    best_avg_new_I = avg_new_Is[np.argmin(RMSE)]
    best_avg_tot_S = avg_tot_Ss[np.argmin(RMSE)]
    best_avg_tot_I = avg_tot_Is[np.argmin(RMSE)]
    best_avg_tot_R = avg_tot_Rs[np.argmin(RMSE)]
        
    print(f"All the sets have been tested! RMSE = {np.min(RMSE)}, params = {param_min_RMSE}")
    return (best_avg_new_I, best_avg_tot_S, best_avg_tot_I, best_avg_tot_R, avg_tot_V)


In [ ]:
vacc = [5, 9, 16, 24, 32, 40, 47, 54, 59, 60, 60, 60, 60, 60, 60, 60]
weekly_infected = [1, 1, 3, 5, 9, 17, 32, 32, 17, 5, 2, 1, 0, 0, 0, 0]
n_simulations = 30

k_range = list(range(7,17))
beta_range = (0.03,0.25)
rho_range = (0.6,0.95)
n_rand_trials = 100
avg_new_I, avg_tot_S, avg_tot_I, avg_tot_R, avg_tot_V = random_search_k_beta_rho(weekly_infected, vacc, a=0, k_range=k_range, beta_range=beta_range, rho_range=rho_range, n_nodes=934, n_rand_trials=n_rand_trials, n_simulations=n_simulations, seed=True)

### Plots
n_weeks = len(vacc)
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_new_I, label='Avg. newly infected (model with best params)')
ax.plot(range(n_weeks), weekly_infected, label='Actual n. of newly infected individuals each week')
plt.setp( ax, xticks=range(n_weeks), xticklabels=list(range(42,53))+list(range(1,6)) )
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()

fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
ax.plot(range(n_weeks), avg_tot_S, label='Avg. total susceptible')
ax.plot(range(n_weeks), avg_tot_I, label='Avg. total infected')
ax.plot(range(n_weeks), avg_tot_R, label='Avg. total recovered')
ax.plot(range(n_weeks), avg_tot_V, label='Avg. total vaccinated')
plt.setp(ax, xticks=range(n_weeks), xticklabels=range(1,n_weeks+1))
ax.set_xlabel('Week', size='large')
ax.set_ylabel('N. of individuals', size='large')
ax.legend(prop={'size': 12})
plt.grid()